<a href="https://colab.research.google.com/github/icanello01/Aula_Agentes_Imersao_IA_Alura/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install --upgrade google-adk

In [35]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [36]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.5-flash-preview-04-17"

In [6]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber sobre as próximas edições é:

*   **Acompanhar as redes sociais da Alura:** Eles costumam anunciar novos eventos no Instagram, LinkedIn e outras plataformas.
*   **Assinar a newsletter da Alura:** Assim, você recebe novidades e informações sobre os cursos e imersões diretamente no seu e-mail.
*   **Verificar a página de cursos e imersões da Alura:** Fique de olho na página oficial para ver se alguma nova edição foi adicionada ao catálogo.

In [7]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Com base nos resultados da pesquisa, a Alura e o Google se uniram para oferecer a Imersão IA com Google Gemini. As inscrições para a Imersão IA são feitas exclusivamente pelo site da Alura até o dia 11 de maio de 2025. Portanto, as inscrições já foram encerradas.

Não há informações sobre uma próxima edição da Imersão IA com Google Gemini da Alura nos resultados da pesquisa. Para saber quando será a próxima edição, você pode acompanhar as redes sociais da Alura ou entrar em contato com a escola.


In [8]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: youtube.com



In [10]:
# Instalar Framework ADK de agentes do Google ################################################
!pip -q install google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [37]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [12]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [13]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [38]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17",
        description="Agente que busca notícias sobre o tópico indicado",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisas. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre eles.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais relevância.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [39]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano
        sobre quais são os pontos mais relevantes que a gente poderia abordar em um
        post sobre cada um dos lançamentos. Você também pode usar o (google_search)
        para encontrar mais informações sobre os temas e se aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base
        nas pesquisas e retornar esse tema, seus pontos mais relevantes, e um
        plano com os assuntos a serem abordados no post que será escrito
        posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [40]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [41]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [43]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"OK! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("######################")
  print("🚀 Lançamentos buscados com sucesso! 🚀")
  print("######################")
  display(to_markdown(lancamentos_buscados))
  print("-----------------------------------------------------------")

  plano_de_post = agente_planejador(topico,lancamentos_buscados)
  print("######################")
  print("🚀 Plano de post gerado com sucesso! 🚀")
  print("######################")
  display(to_markdown(plano_de_post))
  print("-----------------------------------------------------------")

  rascunho_gerado = agente_redator(topico, plano_de_post)
  print("######################")
  print("🚀 Rascunho gerado com sucesso! 🚀")
  print("######################")
  display(to_markdown(rascunho_gerado))
  print("-----------------------------------------------------------")

  post_final = agente_revisor(topico,rascunho_gerado)
  print("######################")
  print("🚀 Texto revisado com sucesso! 🚀")
  print("######################")
  display(to_markdown(post_final))
  print("-----------------------------------------------------------")



🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Funilaria e Pintura de Automóveis
OK! Vamos então criar o post sobre novidades em Funilaria e Pintura de Automóveis
######################
🚀 Lançamentos buscados com sucesso! 🚀
######################


> A busca por lançamentos relevantes em Funilaria e Pintura de Automóveis no último mês (abril/maio de 2025) aponta para algumas áreas de destaque, embora não haja menção a produtos ou tecnologias específicas que se destaquem isoladamente com grande cobertura noticiosa. A relevância no período parece estar mais ligada a eventos do setor e à contínua evolução de técnicas e processos já existentes.
> 
> Com base nas notícias encontradas, os lançamentos mais relevantes parecem ser:
> 
> *   **Foco em Novas Tecnologias e Técnicas de Reparo Moderno:** Há um enfoque significativo na adoção de novas tecnologias para reparos de lataria, como a técnica PDR (Paintless Dent Repair - Reparo Sem Pintura), uso de scanner 3D e medição digital, soldagem por inversão de polaridade e cabines de pintura com controle climático. Essas inovações visam oferecer soluções mais rápidas, econômicas e com acabamento de alto padrão, preservando a pintura original do veículo sempre que possível.
> 
> *   **Produtos de Proteção para Pintura:** O lançamento do Vitrificador Híbrido CC One pela SONAX é noticiado, destacando sua fórmula inovadora que cria uma barreira resistente contra corrosão, sujeira biológica e raios UV, com base em tecnologia de polímeros híbridos.
> 
> *   **Eventos do Setor Automotivo:** As feiras e convenções continuam sendo importantes vitrines para lançamentos e inovações no segmento de funilaria e pintura. Eventos como a Automec Brasil 2025 e a Convenção Bosch 2025 são mencionados como plataformas onde empresas apresentam novidades e tecnologias para oficinas. Feiras internacionais como a AAPEX (que ocorre junto ao SEMA Show) também são relevantes para o segmento de funilaria e pintura, embora o SEMA Show seja tradicionalmente em novembro.
> 
> *   **Tendências em Estética Automotiva e Manutenção:** Além dos reparos, há menções à importância da estética automotiva, incluindo serviços como espelhamento, vitrificação e polimento, e como a manutenção preventiva da pintura é crucial para a preservação do veículo. A adaptação a novas demandas, como a manutenção de carros elétricos e híbridos, também é um ponto relevante para oficinas do setor.
> 
> *   **Robótica na Produção e Reparo:** A robótica continua a influenciar o setor automotivo, incluindo as etapas de pintura, com robôs realizando a aplicação de camadas de tinta de forma uniforme na fabricação.
> 
> Embora as notícias apresentem novidades e a contínua evolução do setor, não identificamos no último mês lançamentos pontuais de produtos ou tecnologias que tenham gerado um volume de notícias ou entusiasmo significativamente maior do que os demais, com exceção talvez do destaque dado às "novas tecnologias" de reparo moderno em Goiânia e ao lançamento do vitrificador da SONAX.


-----------------------------------------------------------
######################
🚀 Plano de post gerado com sucesso! 🚀
######################


> Com base na análise dos lançamentos e tendências recentes em Funilaria e Pintura de Automóveis, e aprofundando a busca com as ferramentas disponíveis, o tema que se destaca como mais relevante e com maior potencial para um post é **"Novas Tecnologias e Técnicas de Reparo Moderno"**.
> 
> Embora não haja um único "lançamento" disruptivo, a constante evolução e a adoção dessas novas abordagens nas oficinas representam o movimento mais significativo no setor, visando maior eficiência, qualidade e satisfação do cliente. As buscas confirmam a relevância de técnicas como o PDR e a importância de ambientes controlados como as cabines de pintura modernas.
> 
> **Tema Mais Relevante:** Novas Tecnologias e Técnicas de Reparo Moderno em Funilaria e Pintura Automotiva.
> 
> **Pontos Mais Relevantes deste Tema:**
> 
> 1.  **Evolução do Reparo:** O setor está migrando de métodos tradicionais para técnicas mais rápidas, menos invasivas e que preservam a originalidade do veículo.
> 2.  **Técnica PDR (Paintless Dent Repair):** Popularmente conhecida como Martelinho de Ouro, é uma técnica não invasiva para remover amassados sem danificar a pintura original, utilizando ferramentas específicas. Seus benefícios incluem menor custo, rapidez no reparo (às vezes no mesmo dia) e preservação do valor de revenda do carro.
> 3.  **Scanner 3D e Medição Digital:** O uso dessas ferramentas garante maior precisão no diagnóstico de danos e no alinhamento da carroceria.
> 4.  **Cabines de Pintura com Controle Climático:** Essenciais para garantir um acabamento de alta qualidade, livre de contaminação por poeira, com controle ideal de temperatura e umidade para a aplicação e cura da tinta. Elas também contribuem para a segurança dos trabalhadores e a eficiência do processo.
> 5.  **Outras Tecnologias:** Incluem soldagem por inversão de polaridade e sistemas de orçamento digital/acompanhamento.
> 6.  **Benefícios para o Cliente:** Reparos mais rápidos, melhor custo-benefício (em muitos casos), manutenção da originalidade e valor do veículo, e um acabamento de maior qualidade.
> 7.  **Adaptação a Novas Demandas:** A necessidade de aplicar essas técnicas e tecnologias também se estende à manutenção de veículos elétricos e híbridos, que possuem requisitos e cuidados específicos, como a sensibilidade das baterias a altas temperaturas em estufas de pintura.
> 
> **Plano de Assuntos para o Post:**
> 
> *   **Título (Sugestão):** Reparo Automotivo do Futuro: As Tecnologias que Estão Transformando a Funilaria e Pintura!
> *   **Introdução:**
>     *   Abordar como o mercado automotivo evoluiu e, com ele, a necessidade de reparos mais sofisticados.
>     *   Apresentar a ideia de "reparo moderno" como um conjunto de técnicas e tecnologias avançadas.
> *   **Seção 1: Adeus, Amassados! A Magia do PDR.**
>     *   Explicar o que é o Paintless Dent Repair (PDR) ou Martelinho de Ouro.
>     *   Mostrar como a técnica funciona de forma não invasiva.
>     *   Listar os principais benefícios: rapidez, economia, preservação da pintura original e do valor do carro.
> *   **Seção 2: Precisão Digital: Scanners 3D e Medição.**
>     *   Explicar como o scanner 3D e a medição digital garantem reparos mais precisos e evitam distorções na estrutura do veículo.
> *   **Seção 3: O Ambiente Ideal: Cabines de Pintura Inteligentes.**
>     *   Destacar a importância de cabines de pintura com controle de temperatura, umidade e filtragem de ar.
>     *   Explicar como essas cabines asseguram um acabamento perfeito, sem contaminações, e uma secagem ideal.
>     *   Mencionar os benefícios de segurança e eficiência.
> *   **Seção 4: Por Que Escolher Oficinas Modernas?**
>     *   Resumir as vantagens para o proprietário do veículo ao optar por serviços que utilizam essas tecnologias (rapidez, qualidade, custo-benefício, valorização do carro).
>     *   Mencionar a necessidade de profissionais qualificados para operar esses equipamentos e técnicas.
> *   **Seção 5 (Opcional/Pequeno Destaque): O Desafio dos Elétricos e Híbridos.**
>     *   Comentar brevemente como a manutenção desses novos veículos exige adaptação e conhecimento técnico na funilaria e pintura.
> *   **Conclusão/Chamada para Ação:**
>     *   Reforçar que investir em reparos modernos é cuidar do seu patrimônio.
>     *   Incentivar o público a buscar oficinas que utilizam essas tecnologias ou a saber mais sobre elas.


-----------------------------------------------------------
######################
🚀 Rascunho gerado com sucesso! 🚀
######################


> Ok, agente redator em ação! 🛠️✨ Vamos transformar esse plano em um rascunho engajador para o Instagram da Alura.
> 
> ---
> 
> **Rascunho de Post para Instagram - Alura**
> 
> **(Idealmente, este post seria um carrossel com imagens representando cada ponto: um carro com amassado antes/depois do PDR, um scanner 3D sendo usado, uma cabine de pintura, um carro moderno/elétrico, etc.)**
> 
> **Slide 1/Imagem de Capa:** Foto impactante de um carro moderno em uma oficina high-tech ou um detalhe de reparo super preciso.
> 
> **Texto da Capa (Opcional na Imagem):** Reparo Automotivo do Futuro: A Era Tech na Funilaria!
> 
> ---
> 
> **Texto do Post:**
> 
> Seu carro amassou ou arranhou? 😩 Esqueça os métodos antigos e demorados! A funilaria e pintura automotiva entrou de vez na era tech e está mudando o jeito que cuidamos do nosso possante! 🚗💨
> 
> Não é mágica, é tecnologia e técnica avançada! As oficinas modernas estão usando ferramentas e processos que transformam o reparo do seu carro. O resultado? Muito mais rapidez, qualidade impecável e a preservação do valor do seu patrimônio. ✅
> 
> **Slide 2:** Imagem ou GIF mostrando a técnica do Martelinho de Ouro (PDR).
> 
> **Texto:** **A Magia do Martelinho de Ouro (PDR): Adeus, Amassados! 🔨➡️✨**
> 
> Aquele amassadinho chato causado por uma porta vizinha ou pedrinha? Nem sempre precisa repintar! O PDR (Paintless Dent Repair) é uma técnica não invasiva que usa ferramentas especiais para "massagear" o metal e remover o defeito sem tocar na pintura original.
> 
> Por que é top?
> 👉 Reparo muito mais rápido (às vezes no mesmo dia!)
> 👉 Custo menor em muitos casos
> 👉 Preserva a pintura de fábrica e o valor do seu carro!
> 
> **Slide 3:** Imagem de um scanner 3D sendo aplicado na lataria de um carro.
> 
> **Texto:** **Precisão de Milímetro com a Ajuda Tech! 📏🔍**
> 
> Na funilaria moderna, não tem "achismo"! Scanners 3D e medições digitais mapeiam o dano com total precisão, garantindo que a estrutura do carro volte ao alinhamento de fábrica e fique perfeita, como se nada tivesse acontecido. Segurança e estética impecáveis!
> 
> **Slide 4:** Imagem de uma cabine de pintura moderna, talvez com um carro dentro.
> 
> **Texto:** **O Ambiente Perfeito para uma Pintura Impecável! 💧💨🎨**
> 
> Quer um acabamento sem poeira, com a cor exata e durabilidade? As cabines de pintura climatizadas são essenciais! Elas controlam temperatura, umidade e filtram o ar, criando o ambiente ideal para a tinta aderir, secar e curar perfeitamente. Qualidade de fábrica e segurança para quem trabalha. 👷‍♀️👷‍♂️
> 
> **Slide 5:** Imagem que combine elementos das tecnologias ou um carro novo/elétrico.
> 
> **Texto:** **Por Que Escolher Oficinas Modernas? 🤔💡**
> 
> Rapidez, qualidade superior, melhor custo-benefício e a certeza de que seu carro está sendo cuidado com o que há de melhor. Investir em reparos modernos é investir no seu patrimônio!
> 
> Mas lembre-se: tecnologia sem um profissional qualificado para usá-la não funciona! A mão de obra especializada e bem treinada é fundamental para operar esses equipamentos e técnicas com maestria. 😉
> 
> **(Opcional/Pode ir no texto principal ou em um slide extra curto):** E até os carros elétricos e híbridos exigem adaptação! Com baterias sensíveis ao calor, as técnicas e ambientes de reparo precisam evoluir. Mais um desafio (e oportunidade!) para o setor! ⚡🚗
> 
> ---
> 
> **Conclusão do Post:**
> 
> Seu carro merece o melhor cuidado tech disponível hoje. Ao escolher uma oficina, procure saber quais tecnologias ela utiliza! ✨
> 
> Curtiu saber sobre essas novidades que estão transformando a funilaria e pintura? Salve este post para lembrar na hora que precisar! 👇
> 
> #FunilariaModerna #TecnologiaAutomotiva #MartelinhoDeOuro #ReparoAutomotivo


-----------------------------------------------------------
######################
🚀 Texto revisado com sucesso! 🚀
######################


> O rascunho está ótimo e pronto para publicar!


-----------------------------------------------------------
